In [ ]:
import requests
from bs4 import BeautifulSoup
from markdownify import markdownify as md

def download_and_convert_to_markdown(url):
    try:
        # Download the webpage content
        response = requests.get(url)
        response.raise_for_status()  # Raises a HTTPError if the response status code is 4XX/5XX
        
        # Parse the HTML
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Extract the text from the 'bt-standard-content' class element
        content = soup.find(class_='bt-standard-content')
        
        # Convert HTML to Markdown
        markdown_text = md(str(content), heading_style="ATX")
        
        return markdown_text
    except requests.exceptions.RequestException as e:
        return f"Error downloading or processing the page: {e}"
    except Exception as e:
        return f"An unexpected error occurred: {e}"

# Example usage
url = "https://www.bundestag.de/ausschuesse/a22_kultur/oeffentliche_sitzungen/976698-976698"
data = download_and_convert_to_markdown(url)


In [ ]:
system_prompt = '''
Du bist ein social media bot mit der Aufgabe die Sitzungen den deutschen Bundestages zusammenzufassen. Du bist extrem gut darin witzig und provokativ nachrichten zu schreiben und dabei trotzdem leute oder gruppen nicht zu beleidigen.
Da du auf social media aktiv bist, weißt du wie man am besten posts vormuliert um für aufsehen zu sorgen. Deine posts sind provokativ und unspezifisch, so dass sich jeder angesprochen fühlt und eine meinung zu dem thema hat. Dein ziel ist es deine posts für eine möglichst große menge an menschen interessant zu machen.
Deine posts sind kurz und knackig. Informationen die nicht direkt mit dem inhalt zu tun haben wie wann und wo die meetings stattgefunden haben werden nicht geteilt.
Thematische genauigkeit ist dir extrem wichtig, und du gibst dir die größte mühe die wahrheit zu schreiben. Wenn etwas nicht eindeutig oder unklar ist, dann sagst du das.
Du bevorzugst keine partei oder politische sichtweise, sondern hast eine offene und positive einstellung. Du kannst zwischen den zeilen lesen und verstehst die politischen motivationen hinter formulierungen. Wenn es sinvoll ist dann erklärst du die hintergründe und motivationen der politischen sprache.
'''

In [ ]:
env = {line.strip().split('=')[0]:line.strip().split('=')[1] for line in open('../.env').read().strip().split('\n') if line.strip()}

from openai import OpenAI

client = OpenAI(
    # This is the default and can be omitted
    api_key=env['OPENAI'],
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": system_prompt.strip(),
        },{
            "role": "user",
            "content": data,
        }
    ],
    model="gpt-3.5-turbo",
)

In [ ]:
print(chat_completion.choices[0].message.content)